Open the bash container:

In [ ]:
img_path=../../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

In [ ]:
img_path=../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

Generate fragments:

In [ ]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8
depth=30k
subset=
for file in ../to_merge/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.FULL.tomerge.tsv}.$depth
    # echo $sample
    jaccardfile=$file
    infile=${depth}_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=${depth}_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    elif [ -f "$outfile2" ]; then
        echo "$outfile2 exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
    fi
done

In [ ]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8
depth=30k
subset=
for file in to_merge/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.FULL.tomerge.tsv}.$depth
    # echo $sample
    jaccardfile=$file
    infile=${depth}/${depth}_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=${depth}/${depth}_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if ! [ -f "$outfile" ]; then
        # echo "$outfile exists, skipping"
        if ! [ -f "$outfile2" ]; then
            echo "$outfile does not exist, generating!"
            merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
        fi
    fi
done

Then bgzip the output:

In [ ]:
module load HTSlib
depth=30k
for file in ${depth}_preprocessing_out/data/fragments/*$subset*.fragments.tsv
do
    echo $file
    if [ -f "$file.gz" ]; then
        echo exists
    else
        bgzip -@ 4 $file &
    fi
done

In [ ]:
module load HTSlib
for file in *k/*_preprocessing_out/data/fragments/*$subset*.fragments.tsv
do
    echo $file
    if [ -f "$file.gz" ]; then
        echo exists
    else
        bgzip -@ 4 $file &
    fi
done

In [ ]:
jobs

and tabix index

In [ ]:
module load HTSlib
for file in ${depth}_preprocessing_out/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done

In [ ]:
module load HTSlib
for file in *k/*_preprocessing_out/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done

In [ ]:
jobs